In [ ]:
import os
from collections import Counter
import unidecode
import unicodedata
from unidecode import unidecode_expect_ascii
import gc
import json
import re
from itertools import chain
ROOT = 'ORIGINAL_CSV'
# ROOT = 'CSV'

def get_dodf_type(s, dbg=False, default='integra'):
    s = s.lower()
    if 'sup' in s:
        return 'suplemento'
    elif 'ext' in s or 'etx' in s or 'exr' in s:
        return 'extra'
    elif 'esp' in s:
        return 'especial'
    elif 'int' in s or 'sec' in s:
        return 'integra'
    else:
        if dbg:
            print("SEM CLASSE:", s, '| default: ', default)        
        return default

In [ ]:
%%time
# `diretorios`: lista dos DIRETÓRIOS relativos aos DODFs.
# Não há portanto duplicatas, mas ocorre de vários elementos 
# possuírem mesmo prefixo.

diretorios = [i[0] for i in os.walk(ROOT)]
nempty = [i for i in os.walk(ROOT) if (i[1] or i[2])]
empty = [i[0] for i in os.walk(ROOT) if (not i[1] and not i[2])]

print("paths, not-empty, empty")
print(len(diretorios), len(nempty), len(empty), end='\n\n')
del empty, nempty;
gc.collect();

## Note-se que afinal, as primeiras 3 partes de QUALQUER diretório são dispensáveis para o fim de contagem, servindo apenas para organização

In [ ]:
hier = {}
def prefix(x):
    return '/'.joins(x.split('/')[:-1])
for s in diretorios:
    l = s.split('/')
    d = hier
    for p in l[:-1]:
        d = d.get(p, dict())
    d[l[-1]] = dict()
hier[ROOT].keys()

In [ ]:
diretorios_por_ano = {}
for year in hier[ROOT]:
    diretorios_por_ano[year] = [
        i for i in diretorios if year in i and i.count('/') > 2
    ]
# diretorios_por_ano['2001'][:10]

In [ ]:
Counter([i.count('/') for i in diretorios ])

In [ ]:
hier = {}
# def normalize_dir(s):
#     s = unidecode_expect_ascii(s)
#     return s.lower()

worthy_dir = [i for i in diretorios if i.count('/') > 2]
# has_deg = []
for i in sorted(
#     map(normalize_dir, worthy_dir),
    worthy_dir,
    key=lambda x: (x.count('/'), re.search(r'\d{4}', x).group())):
    qtd_barra = i.count('/')
#     if 'deg' in i:
#     if '°' in i:
        
#         """"DECISÃO: ignorar esses raríssimos casos"""
#         has_deg.append(i)
#         continue
        
    if qtd_barra == 3:
        if i in hier:
            print("CONFLITO:", i)
        hier[i] = []
#   qtd_barra == 4 ou seja: é um arquivo de uma pasta
#   com várias partes de um DODF
    else:
        hier[ i[:i.rfind('/')] ].append(i.split('/')[-1])
# print("Tem 'deg' no nome:", len(has_deg))
# """Como são muito poucos, serão ignorados"""

In [ ]:
Counter({k:get_dodf_type(k) for (k,v) in hier.items()}.values())

In [ ]:
json.dump(hier, open('hierarquia_dodfs.json', 'w', encoding='utf8'),
          indent=4*' ', ensure_ascii=False)